In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv("D:/1.năm 4/Dự án AI/archive/digital_marketing_campaign_dataset.csv")
df.head()

,CustomerID,Age,Gender,Income,CampaignChannel,CampaignType,AdSpend,ClickThroughRate,ConversionRate,WebsiteVisits,PagesPerVisit,TimeOnSite,SocialShares,EmailOpens,EmailClicks,PreviousPurchases,LoyaltyPoints,AdvertisingPlatform,AdvertisingTool,Conversion
0,8000,56,Female,136912,Social Media,Awareness,6497.870068,0.043919,0.088031,0,2.399017,7.396803,19,6,9,4,688,IsConfid,ToolConfid,1
1,8001,69,Male,41760,Email,Retention,3898.668606,0.155725,0.182725,42,2.917138,5.352549,5,2,7,2,3459,IsConfid,ToolConfid,1
2,8002,46,Female,88456,PPC,Awareness,1546.429596,0.277490,0.076423,2,8.223619,13.794901,0,11,2,8,2337,IsConfid,ToolConfid,1
3,8003,32,Female,44085,PPC,Conversion,539.525936,0.137611,0.088004,47,4.540939,14.688363,89,2,2,0,2463,IsConfid,ToolConfid,1
4,8004,60,Female,83964,PPC,Conversion,1678.043573,0.252851,0.109940,0,2.046847,13.993370,6,6,6,8,4345,IsConfid,ToolConfid,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           8000 non-null   int64  
 1   Age                  8000 non-null   int64  
 2   Gender               8000 non-null   object 
 3   Income               8000 non-null   int64  
 4   CampaignChannel      8000 non-null   object 
 5   CampaignType         8000 non-null   object 
 6   AdSpend              8000 non-null   float64
 7   ClickThroughRate     8000 non-null   float64
 8   ConversionRate       8000 non-null   float64
 9   WebsiteVisits        8000 non-null   int64  
 10  PagesPerVisit        8000 non-null   float64
 11  TimeOnSite           8000 non-null   float64
 12  SocialShares         8000 non-null   int64  
 13  EmailOpens           8000 non-null   int64  
 14  EmailClicks          8000 non-null   int64  
 15  PreviousPurchases    8000 non-null   i

## Kiểm tra dữ liệu thiếu và xử lý 

In [7]:
df.isnull().sum()

CustomerID             0
Age                    0
Gender                 0
Income                 0
CampaignChannel        0
CampaignType           0
AdSpend                0
ClickThroughRate       0
ConversionRate         0
WebsiteVisits          0
PagesPerVisit          0
TimeOnSite             0
SocialShares           0
EmailOpens             0
EmailClicks            0
PreviousPurchases      0
LoyaltyPoints          0
AdvertisingPlatform    0
AdvertisingTool        0
Conversion             0
dtype: int64

In [9]:
numeric_columns = df.select_dtypes(include=['number']).columns

In [11]:
### 1. Kiểm tra Outliers bằng IQR
# Tính Q1 (25th percentile) và Q3 (75th percentile)
Q1 = df[numeric_columns].quantile(0.25)
Q3 = df[numeric_columns].quantile(0.75)
IQR = Q3 - Q1

# Xác định các dòng có outliers dựa trên IQR
outliers_iqr_condition = (df[numeric_columns] < (Q1 - 1.5 * IQR)) | (df[numeric_columns] > (Q3 + 1.5 * IQR))
outliers_iqr_count = outliers_iqr_condition.any(axis=1).sum()

### 2. Kiểm tra Outliers bằng Z-score
from scipy import stats
# Tính Z-score
z_scores = np.abs(stats.zscore(df[numeric_columns]))

# Xác định các dòng có outliers dựa trên Z-score (Z-score > 3)
outliers_z_condition = (z_scores > 3)
outliers_z_count = outliers_z_condition.any(axis=1).sum()

# Hiển thị kết quả
print(f'Số lượng dòng có outliers (theo IQR): {outliers_iqr_count}')
print(f'Số lượng dòng có outliers (theo Z-score): {outliers_z_count}')

Số lượng dòng có outliers (theo IQR): 988
Số lượng dòng có outliers (theo Z-score): 0


In [13]:
df = df.drop(['CustomerID', 'AdvertisingPlatform', 'AdvertisingTool'], axis=1)

## Chuyển đổi dữ liệu 

In [16]:
from sklearn.preprocessing import LabelEncoder
ob_features = df.select_dtypes(include = 'object').columns

# Khởi tạo bộ mã hóa nhãn
le = LabelEncoder()

# Áp dụng bộ mã hóa nhãn cho các cột thuộc loại dữ liệu categorical
df[ob_features] = df[ob_features].apply(lambda col: le.fit_transform(col))

In [19]:
df.info

<bound method DataFrame.info of       Age  Gender  Income  CampaignChannel  CampaignType      AdSpend  \
0      56       0  136912                4             0  6497.870068   
1      69       1   41760                0             3  3898.668606   
2      46       0   88456                1             0  1546.429596   
3      32       0   44085                1             2   539.525936   
4      60       0   83964                1             2  1678.043573   
...   ...     ...     ...              ...           ...          ...   
7995   21       1   24849                0             0  8518.308575   
7996   43       0   44718                3             3  1424.613446   
7997   28       0  125471                2             1  4609.534635   
7998   19       0  107862                1             1  9476.106354   
7999   31       0   93002                0             0  7743.627070   

      ClickThroughRate  ConversionRate  WebsiteVisits  PagesPerVisit  \
0             0.043

## Chia dữ liệu

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
# Chia dữ liệu thành features (X) và target (y)
X = df.drop(['Conversion'], axis=1)
y = df['Conversion']
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Mô hình KNN

In [21]:
from sklearn.neighbors import KNeighborsClassifier
# Khởi tạo mô hình KNN
knn_model = KNeighborsClassifier(n_neighbors= 113, weights = 'uniform', metric = 'manhattan')

# Huấn luyện mô hình trên tập huấn luyện
knn_model.fit(X_train, y_train)
# Dự đoán trên tập kiểm tra
y_pred = knn_model.predict(X_test)
# Đánh giá hiệu suất của mô hình
print('MÔ HÌNH K-Nearest Neighbors":')
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy *100:.2f}' ' %')

# In ra confusion matrix và báo cáo classification
print('\nClassification Report:')
print(classification_report(y_test, y_pred))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_pred))

MÔ HÌNH K-Nearest Neighbors":
Accuracy: 87.88 %

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       194
           1       0.88      1.00      0.94      1406

    accuracy                           0.88      1600
   macro avg       0.44      0.50      0.47      1600
weighted avg       0.77      0.88      0.82      1600


Confusion Matrix:
[[   0  194]
 [   0 1406]]


C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Mô hình hồi quy logitic

In [24]:
from sklearn.linear_model import LogisticRegression

print('Kết quả mô hình hồi quy logitic:')

# Khởi tạo mô hình Logistic Regression
logreg_model = LogisticRegression(max_iter = 1000)

# Huấn luyện mô hình trên tập huấn luyện
logreg_model.fit(X_train, y_train)

# Dự đoán nhãn của dữ liệu kiểm tra
y_pred = logreg_model.predict(X_test)
# Đánh giá hiệu suất của mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100 :.2f}'' %')

# In ra confusion matrix và báo cáo classification
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Kết quả mô hình hồi quy logitic:
Accuracy: 87.94 %
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.01      0.01       194
           1       0.88      1.00      0.94      1406

    accuracy                           0.88      1600
   macro avg       0.94      0.50      0.47      1600
weighted avg       0.89      0.88      0.82      1600

Confusion Matrix:
[[   1  193]
 [   0 1406]]


C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
# Lấy ra các hệ số (weights) và intercept từ mô hình
weights = logreg_model.coef_
intercept =logreg_model.intercept_

# In ra hàm dự đoán
print("Hàm dự đoán:")
print(f"y_pred = sigmoid({intercept} + {weights} * X)")

Hàm dự đoán:
y_pred = sigmoid([-0.01049566] + [[-1.15350117e-02 -4.72626165e-03 -3.19025792e-06 -3.00071351e-02
   8.07069211e-03  9.28264617e-05  5.19892958e-03  2.33901511e-03
   8.05689061e-03  6.67140193e-02  6.96089486e-02 -5.65929559e-03
   5.79385685e-02  1.10228995e-01  8.61817741e-02  1.43708824e-04]] * X)


## Mô hình Random forest

In [30]:
from sklearn.ensemble import RandomForestClassifier

# Khởi tạo mô hình Random Forest
rf_model = RandomForestClassifier()

# Huấn luyện mô hình trên tập huấn luyện
rf_model.fit(X_train, y_train)

# Dự đoán nhãn của dữ liệu kiểm tra
y_pred = rf_model.predict(X_test)

# Đánh giá mô hình
print('Kết quả mô hình Random forest:')
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy *100:.2f}' ' %')

# In ra báo cáo phân loại và ma trận nhầm lẫn
print('Classification Report:')
print(classification_report(y_test, y_pred))

print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Kết quả mô hình Random forest:
Accuracy: 89.81 %
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.21      0.33       194
           1       0.90      0.99      0.94      1406

    accuracy                           0.90      1600
   macro avg       0.86      0.60      0.64      1600
weighted avg       0.89      0.90      0.87      1600

Confusion Matrix:
[[  40  154]
 [   9 1397]]


## Mô hình XGBoost

In [32]:
from xgboost import XGBClassifier
# Xây dựng mô hình XGBoost
xgboost_model = XGBClassifier()
xgboost_model.fit(X_train, y_train)

# Dự đoán trên tập test
y_pred = xgboost_model.predict(X_test)

# Đánh giá hiệu suất của mô hình
print('Kết quả mô hình XGBoost":')
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}')

# In ra confusion matrix và báo cáo classification
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Kết quả mô hình XGBoost":
Accuracy: 90.94
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.39      0.51       194
           1       0.92      0.98      0.95      1406

    accuracy                           0.91      1600
   macro avg       0.83      0.68      0.73      1600
weighted avg       0.90      0.91      0.90      1600

Confusion Matrix:
[[  75  119]
 [  26 1380]]


In [37]:
# 4. Xây dựng mô hình ANN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
model = Sequential()

# Input layer và hidden layers
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))  # Lớp 1 với 64 neurons
model.add(Dropout(0.5))  # Dropout để giảm overfitting
model.add(Dense(32, activation='relu'))  # Lớp 2 với 32 neurons
model.add(Dense(16, activation='relu'))  # Lớp 3 với 16 neurons

# Output layer
model.add(Dense(1, activation='sigmoid'))  # 1 neuron cho bài toán phân loại nhị phân

# 5. Compile mô hình
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 6. Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# 7. Đánh giá mô hình
# Dự đoán trên tập test
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# In báo cáo Classification Report và Confusion Matrix
print('Kết quả mô hình ANN:')
print(f"Accuracy: {accuracy:.2%}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# 8. Đánh giá độ chính xác
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

C:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.7898 - loss: 772.1320 - val_accuracy: 0.8788 - val_loss: 61.2901
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7825 - loss: 91.9852 - val_accuracy: 0.8788 - val_loss: 0.6362
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8401 - loss: 15.5874 - val_accuracy: 0.8788 - val_loss: 0.5496
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8646 - loss: 4.3570 - val_accuracy: 0.8788 - val_loss: 0.4933
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8685 - loss: 1.7411 - val_accuracy: 0.8788 - val_loss: 0.4568
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8736 - loss: 1.0805 - val_accuracy: 0.8788 - val_loss: 0.4316
Epoch 7/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8801 - loss: 1.1327 - val_accuracy: 0.8788 - val_loss: 0.4134
Epoch 8/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8680 - loss: 0.5416 - val_accur

C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
